## Modelo creado con la finalidad de encontrar secunecias de ADN que distingan entre 3 especies distintas

#### Librerías

Las librerías utilizadas son:  

- **Torch**:  
  Se utiliza como base para la construcción del modelo. Su función es la de construir y entrenar redes neuronales.  

- **Torch-geometric**:  
  Da herramientas para la manipulación de datos, así como generar los grafos.  

- **Random**:  
  Permite generar números random y, a su vez, también nos permite utilizar funciones con algunos valores random.  


In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import random


 **with open("/home/crakerool/datasets/chimpanzee.txt", "r") as file:**  
**with open()**: Permite acceder/abrir un archivo, y este se cerrará automáticamente al finalizar la operación.  
**Ruta**: Explica dónde se encuentra el archivo en cuestión.  
**r**: Esto indica que el archivo solo se puede acceder en modo lectura, sin permitir la modificación.  
**as file**: Esto indica que se está asignando el archivo a la variable **file**, lo que nos permitirá interactuar con el archivo más adelante.  

**chimpanzee_sequences = [line.strip() for line in file if line.strip()]**  
 **variable =**: Asigna lo que esté dentro de esta expresión a una variable.  
 **[line.strip() for line in file]**:  
 **line.strip()**: Elimina los espacios vacíos al inicio y al final de la línea. También incluye la eliminación de saltos de línea representados como **\n**.  
 **for line in file**: Indica que se realizará la acción en cada línea del archivo.  
**if line.strip()**: Condicional que indica que solo se procesarán las líneas que no estén vacías.  


In [2]:
# Cargar las secuencias desde los archivos FASTA y asignar etiquetas
with open("/home/crakerool/datasets/chimpanzee.txt", "r") as file:
    chimpanzee_sequences = [line.strip() for line in file if line.strip()]
with open("/home/crakerool/datasets/dog.txt", "r") as file:
    dog_sequences = [line.strip() for line in file if line.strip()]
with open("/home/crakerool/datasets/human.txt", "r") as file:
    human_sequences = [line.strip() for line in file if line.strip()]

Parte del codigo que indica que el el codigo se cargo correctamente por medio de una condicional

In [3]:
# Verificar que se hayan cargado secuencias
if not chimpanzee_sequences:
    raise ValueError("No se encontraron secuencias en el archivo chimpanzee.txt")
if not dog_sequences:
    raise ValueError("No se encontraron secuencias en el archivo dog.txt")
if not human_sequences:
    raise ValueError("No se encontraron secuencias en el archivo human.txt")


**min_count = min(len(chimpanzee_sequences), len(dog_sequences), len(human_sequences))**  
**min()**: Toma el valor más pequeño dentro de los proporcionados.  
**len()**: Lee la cantidad de secuencias dentro del archivo.  
Toma la mínima cantidad de secuencias como un valor de la variable **min_count**.



In [4]:
# Definir la cantidad mínima de secuencias entre todas las clases para balancear el dataset
min_count = min(len(chimpanzee_sequences), len(dog_sequences), len(human_sequences))

**variable = random.sample(secuencia_variable, min_count)**    
**random.sample(poblacion, z)**: Esto toma **z** como valores aleatorios de la variable **población**.

In [5]:
# Submuestreo: reducir cada clase a la misma cantidad de secuencias que la clase minoritaria
balanced_chimpanzee_sequences = random.sample(chimpanzee_sequences, min_count)
balanced_dog_sequences = random.sample(dog_sequences, min_count)
balanced_human_sequences = random.sample(human_sequences, min_count)

**variable = [z] * min_count**  
**z**: La lista que se cree tendrá el valor **z**.  
Lo que se añadirá dependerá de cuántos valores existan en **min_count**.




In [6]:
# Crear las etiquetas correspondientes
balanced_chimpanzee_labels = [0] * min_count
balanced_dog_labels = [1] * min_count
balanced_human_labels = [2] * min_count

Suma las distintas variables con sus respetivas cosas

In [7]:
# Combinar las secuencias y etiquetas balanceadas
balanced_sequences = balanced_chimpanzee_sequences + balanced_dog_sequences + balanced_human_sequences
balanced_labels = balanced_chimpanzee_labels + balanced_dog_labels + balanced_human_labels

**combined = list(zip(balanced_sequences, balanced_labels))**  
**list**: Indica que se está tratando con listas y seguirá en formato de listas.



**combined = list(zip(balanced_sequences, balanced_labels))**  
**combined**: Asigna una nueva variable que contiene los elementos combinados de dos listas.  
**zip()**: Combina dos o más listas, creando pares (tuplas) donde cada elemento de una lista se empareja con el correspondiente de la otra lista.  
**list()**: Convierte el resultado de **zip()** en una lista para que sea manipulable.

**random.shuffle(combined)**  
Reordena aleatoriamente los elementos de la lista proporcionada, esto de la variable **combined**

**balanced_sequences, balanced_labels = zip(combined)**  
Separa las listas en dos distintas sin perder sus conexiones.



In [8]:
# Mezclar las secuencias y etiquetas para que estén en un orden aleatorio
combined = list(zip(balanced_sequences, balanced_labels))
random.shuffle(combined)
balanced_sequences, balanced_labels = zip(*combined)

**def encode_sequence(sequence)**  
**def**: Define una función con el nombre especificado.  
**encode**: Es el nombre de la función.  
**()**: Dentro de los paréntesis se toma la variable que representa lo que será definido.

**encoding = {'A': 0, 'T': 1, 'C': 2, 'G': 3}**  
**encoding**: Variable que convierte los parámetros a sus respectivos valores numéricos.

**return [encoding[base] for base in sequence if base in encoding]**    
**[encoding[base] for base in sequence]**: Utiliza una comprensión de listas para procesar cada carácter (base) de la secuencia.  
**for base in sequence**: Itera sobre cada base en la secuencia.  
**encoding[base]**: Usa el diccionario **encoding** para obtener el valor numérico asociado a la base.  
**if base in encoding**: Filtra las bases, asegurándose de que solo se procesen las que están en el diccionario 
**encoding**. Si la secuencia contiene caracteres no válidos (como espacios o letras diferentes), estos serán ignorados.  



In [9]:
# Codificar las secuencias como vectores numéricos
def encode_sequence(sequence):
    encoding = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
    return [encoding[base] for base in sequence if base in encoding]

**def sequence_to_graph(sequence)**
Se crea una nueva función de Python sobre la "base" de secuencia, es decir, se usa la secuencia como parámetro de entrada para actuar sobre ella.

**x = torch.tensor(sequence, dtype=torch.float).view(-1, 1)**
**torch.tensor(sequence)**: Convierte la secuencia numérica en un tensor de PyTorch para que sea compatible con modelos basados en PyTorch.  
**dtype=torch.float**: Especifica que los valores serán de tipo flotante.  
**.view(-1, 1)**: Cambia la forma (dimensiones) del tensor para que cada número sea un nodo con un solo atributo.  
**-1**: Ajusta automáticamente el tamaño de la primera dimensión (número de nodos), lo cual significa que las listas tendrán una única forma específica, por ejemplo:  
[0,  
1,  
2,  
3].

**1**: Especifica que cada nodo tendrá un único atributo (el valor del nucleótido).





**edge_index = []**  
Crea una lista vacía que almacenará las conexiones (bordes) entre los nodos.
 
**for i in range(len(sequence) - 1)**  
Itera sobre cada índice de la secuencia, excepto el último, porque los bordes conectan nodos consecutivos.

**edge_index.append([i, i + 1])**  
Agrega un borde dirigido desde el nodo **i** al nodo **i + 1**.

**edge_index.append([i + 1, i])**  
Agrega un borde dirigido desde el nodo **i + 1** al nodo **i** (haciendo la conexión bidireccional).

**edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()**  
**torch.tensor(edge_index, dtype=torch.long)**: Convierte la lista **edge_index** en un tensor de tipo entero largo (**long**), requerido para representar índices de nodos en PyTorch.  
**.t()**: Transpone el tensor para que los bordes estén en el formato correcto, donde cada columna representa un borde (**[nodo_origen, nodo_destino]**).  
**.contiguous()**: Asegura que el tensor sea continuo en memoria, lo cual es necesario para operaciones eficientes en PyTorch.   



**return Data(x=x, edge_index=edge_index)**    
crea el grafo 

In [10]:
# Convertir una secuencia codificada en un grafo
def sequence_to_graph(sequence):
    x = torch.tensor(sequence, dtype=torch.float).view(-1, 1)  # Nodos con un solo atributo (el valor del nucleótido)
    edge_index = []
    for i in range(len(sequence) - 1):
        edge_index.append([i, i + 1])
        edge_index.append([i + 1, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return Data(x=x, edge_index=edge_index)



  **graph_list = []**
Crea una lista vacía para almacenar los grafos generados.



  **for i, seq_record in enumerate(balanced_sequences)**
Itera sobre las secuencias balanceadas junto con sus índices.



  **encoded_sequence = encode_sequence(seq_record)**
Codifica cada secuencia en una lista numérica.



  **if len(encoded_sequence) == 0**
Omite secuencias vacías.



  **graph = sequence_to_graph(encoded_sequence)**
Convierte la secuencia codificada en un grafo.



  **graph.y = balanced_labels[i]**
Asigna la etiqueta correspondiente (**balanced_labels[i]**) al grafo.



  **graph_list.append(graph)**
Agrega el grafo procesado a la lista.



In [11]:
# Crear los grafos a partir de las secuencias balanceadas
graph_list = []
for i, seq_record in enumerate(balanced_sequences):
    encoded_sequence = encode_sequence(seq_record)
    if len(encoded_sequence) == 0:
        continue  # Omitir secuencias vacías
    graph = sequence_to_graph(encoded_sequence)
    graph.y = torch.tensor([balanced_labels[i]], dtype=torch.long)  # Asignar la etiqueta correspondiente al organismo
    graph_list.append(graph)

In [12]:
# Verificar que se hayan creado grafos
if len(graph_list) == 0:
    raise ValueError("No se pudieron crear grafos a partir de las secuencias proporcionadas.")


**class ImprovedGNNModel(torch.nn.Module)**  
Declara una clase que hereda de **torch.nn.Module**, el estándar para modelos en PyTorch. Modelo utilizado al momento de hacer GNN.

**def __init__(self, num_classes)**  
Define los componentes en la función **__init__**.

**self.conv1 = GCNConv(1, 32)**
**self.conv2 = GCNConv(32, 64)**
**self.conv3 = GCNConv(64, 128)**
**GCNConv**: Implementa una convolución para grafos usando **torch-geometric**. La entrada de cada capa es el número de características de los nodos en la capa anterior. La salida de cada capa es el número de características aprendidas.
  - **Primera capa (conv1)**: Toma 1 característica por nodo (por ejemplo, el valor numérico del nucleótido). Produce 32 características por nodo.
  - **Segunda capa (conv2)**: Toma 32 características y produce 64 características.
  - **Tercera capa (conv3)**: Toma 64 características y produce 128 características.

**self.dropout = torch.nn.Dropout(p=0.5)**
**Dropout**: Apaga aleatoriamente el 50% (**p=0.5**) de las neuronas durante el entrenamiento.

**self.lin = torch.nn.Linear(128, num_classes)**
**torch.nn.Linear**: Toma las 128 características por grafo producidas por las capas convolucionales. Genera una salida de tamaño igual a **num_classes** (una predicción para cada clase).



**def forward(self, data)**  
Define el modelo.

**x, edge_index = data.x, data.edge_index**  
Extrae las características de los nodos (**x**) y las conexiones entre ellos (**edge_index**) desde el objeto **data**.

**x = F.relu(self.conv1(x, edge_index))**  
**Primera capa**: Aplica la convolución **conv1** para generar características por nodo. Dependiendo de la capa, la cantidad será diferente (32 en este caso).  
**F.relu**: Aplica la función de activación **ReLU**, que introduce no linealidad al modelo.

**x = self.dropout(x)**  
**Dropout**: Apaga algunas características de forma aleatoria, mejorando la generalización.

**x = F.relu(self.conv3(x, edge_index))**  
Similar a lo anterior, pero aplicado en la **tercera capa**.

**x = global_mean_pool(x, data.batch)**  
**global_mean_pool**: Reduce las características de todos los nodos en un único vector para cada grafo.  
**data.batch**: Agrupa nodos del mismo grafo para calcular un promedio por grafo.

**x = self.lin(x)**  
Pasa el vector del grafo por la capa lineal para producir predicciones de tamaño **num_classes**.

**return F.log_softmax(x, dim=-1)**  
**F.log_softmax**: Convierte las salidas en probabilidades logarítmicas, que son útiles para clasificación multi-clase.







In [13]:
class ImprovedGNNModel(torch.nn.Module):
    def __init__(self, num_classes):
        super(ImprovedGNNModel, self).__init__()
        self.conv1 = GCNConv(1, 32)  # Incrementar el número de características
        self.conv2 = GCNConv(32, 64)
        self.conv3 = GCNConv(64, 128)  # Añadir una tercera capa de convolución
        self.dropout = torch.nn.Dropout(p=0.5)  # Dropout para regularización
        self.lin = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)  # Aplicar dropout después de la segunda capa
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, data.batch)  # Pooling global para obtener una representación del grafo completo
        x = self.lin(x)
        return F.log_softmax(x, dim=-1)

**torch.device**
Determina si el modelo usará **GPU (cuda)** o **CPU**, dependiendo de la disponibilidad.

**num_classes**
Calcula el número de clases únicas para clasificar los datos.

**ImprovedGNNModel**
Instancia el modelo GNN configurado con el número correcto de clases y lo mueve al dispositivo seleccionado.

**torch.optim.Adam**
Optimizador que ajusta los parámetros del modelo durante el entrenamiento con una tasa de aprendizaje específica.

**DataLoader**
Organiza y prepara los datos para ser procesados en lotes (**batch**) durante el entrenamiento, con mezcla aleatoria si se requiere.

**model.train()**
Configura el modelo en modo de entrenamiento (activa **dropout** y otras características).

**optimizer.zero_grad()**
Limpia gradientes acumulados de iteraciones previas.

**model(data)**
Genera predicciones del modelo para los datos actuales.

**F.nll_loss**
Calcula la pérdida comparando las predicciones con las etiquetas verdaderas.

**loss.backward()**
Calcula los gradientes de los parámetros del modelo respecto a la pérdida.

**optimizer.step()**
Ajusta los parámetros del modelo usando los gradientes calculados.



In [14]:
# Preparar el entrenamiento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(set(balanced_labels))  # Número de clases, correspondiente al número de organismos
model = ImprovedGNNModel(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loader = DataLoader(graph_list, batch_size=16, shuffle=True)

# Entrenar el modelo
def train():
    model.train()
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

/home/crakerool/miniconda3/envs/gnn_env/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


**Ciclo de entrenamiento (for epoch in range(1, 500))**
Ejecuta el proceso de entrenamiento por 500 iteraciones o épocas. Cada época representa una pasada completa sobre los datos.

**Entrenamiento del modelo (train())**
Llama a la función de entrenamiento definida previamente, que ajusta los parámetros del modelo en función de los datos y las etiquetas.

**Progreso del entrenamiento (if epoch % 10 == 0)**
Imprime un mensaje cada 10 épocas para indicar el progreso del entrenamiento.

Al completar todas las épocas, muestra un mensaje que confirma que el proceso ha terminado.




In [15]:
# Evaluar el modelo (opcional, agregar función de evaluación)
for epoch in range(1, 500):
    train()
    if epoch % 10 == 0:
        print(f"Epoch {epoch} completed.")

print("Entrenamiento completado.")


Epoch 10 completed.
Epoch 20 completed.
Epoch 30 completed.
Epoch 40 completed.
Epoch 50 completed.
Epoch 60 completed.
Epoch 70 completed.
Epoch 80 completed.
Epoch 90 completed.
Epoch 100 completed.
Epoch 110 completed.
Epoch 120 completed.
Epoch 130 completed.
Epoch 140 completed.
Epoch 150 completed.
Epoch 160 completed.
Epoch 170 completed.
Epoch 180 completed.
Epoch 190 completed.
Epoch 200 completed.
Epoch 210 completed.
Epoch 220 completed.
Epoch 230 completed.
Epoch 240 completed.
Epoch 250 completed.
Epoch 260 completed.
Epoch 270 completed.
Epoch 280 completed.
Epoch 290 completed.
Epoch 300 completed.
Epoch 310 completed.
Epoch 320 completed.
Epoch 330 completed.
Epoch 340 completed.
Epoch 350 completed.
Epoch 360 completed.
Epoch 370 completed.
Epoch 380 completed.
Epoch 390 completed.
Epoch 400 completed.
Epoch 410 completed.
Epoch 420 completed.
Epoch 430 completed.
Epoch 440 completed.
Epoch 450 completed.
Epoch 460 completed.
Epoch 470 completed.
Epoch 480 completed.
E

#### Resultados
Al observar los resultados que obtenemos, se observa que a diferencia de antes que dava 2 constantemente ahora genera una variante de esto lo que indica un avanze.


In [16]:
# Evaluar el modelo y mostrar algunos resultados
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)  # Predicción de la clase con mayor probabilidad
        correct += (pred == data.y).sum().item()
        total += data.num_graphs

    accuracy = correct / total
    print(f'Precisión en el conjunto de entrenamiento: {accuracy * 100:.2f}%')

# Mostrar algunas predicciones de ejemplo
for i, data in enumerate(loader):
    if i >= 5:  # Mostrar solo 5 ejemplos
        break
    data = data.to(device)
    out = model(data)
    pred = out.argmax(dim=1)
    print(f"Secuencia real: {data.y.tolist()}, Predicción del modelo: {pred.tolist()}")

Precisión en el conjunto de entrenamiento: 37.04%
Secuencia real: [0, 2, 0, 2, 1, 1, 2, 0, 2, 0, 2, 0, 0, 1, 1, 1], Predicción del modelo: [2, 0, 2, 1, 2, 0, 2, 0, 2, 0, 0, 1, 2, 0, 1, 2]
Secuencia real: [0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0, 2, 2, 0, 0], Predicción del modelo: [0, 1, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 2, 0, 0]
Secuencia real: [2, 0, 2, 1, 2, 1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 2], Predicción del modelo: [1, 0, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 2, 2, 0, 0]
Secuencia real: [1, 2, 2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 2, 2, 1], Predicción del modelo: [0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 1, 0, 1, 1]
Secuencia real: [1, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 0, 2, 0, 1, 1], Predicción del modelo: [2, 0, 0, 2, 0, 2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0]
